In [1]:
import re
from ast import literal_eval

import pandas as pd
from tqdm.notebook import tqdm
from nltk import word_tokenize

In [2]:
tqdm.pandas()

In [3]:
df_train = pd.read_csv("../data/train_v0.7.1.csv",
                       usecols=[
                           'entry_id', 'excerpt', 'gender', 'lang',
                           "translation_en", "translation_fr", "translation_es"
                       ])
df_val = pd.read_csv("../data/val_v0.7.1.csv",
                     usecols=[
                         'entry_id', 'excerpt', 'gender', 'lang',
                         "translation_en", "translation_fr", "translation_es"
                     ])
df_test = pd.read_csv("../data/test_v0.7.1.csv",
                      usecols=[
                          'entry_id', 'excerpt', 'gender', 'lang',
                          "translation_en", "translation_fr", "translation_es"
                      ])

In [4]:
df_test

,entry_id,excerpt,gender,lang,translation_en,translation_fr,translation_es
0,16851,"During the reporting week, IOM provided medica...",[],en,NaN,"Au cours de la semaine de reporting, l'OIM a f...","Durante la semana de presentación de informes,..."
1,489433,Primary and secondary net enrollment rates are...,[],en,NaN,Les taux de scolarisation nets primaires et se...,Las tasas de inscripción netas primarias y sec...
2,186155,Impacto en las personas a. Inseguridad aliment...,[],es,Impact on people a. Food insecurity Recurrent ...,Impact sur les gens a. Insécurité alimentaire ...,NaN
3,187037,"• According to the World Bank, most Venezuelan...",[],en,NaN,"• Selon la Banque mondiale, la plupart des mig...","• Según el Banco Mundial, la mayoría de los mi..."
4,489430,"Like few other countries globally, the majorit...",[],en,NaN,"Comme peu d'autres pays à l'échelle mondiale, ...",Al igual que algunos otros países a nivel mund...
...,...,...,...,...,...,...,...
17195,347034,S’il s’agit d’un mouvement spontané au lieu d’...,[],fr,In the case of a spontaneous movement instead ...,NaN,En el caso de un movimiento espontáneo en luga...
17196,347041,"En raison de la forte demande, une augmentatio...",[],fr,"Due to the strong demand, an increase in the p...",NaN,"Debido a la fuerte demanda, a veces se ha obse..."
17197,347063,"D’après les études MSA des équipes RRM, le niv...",[],fr,"According to MSA studies of RRM teams, the lev...",NaN,"Según los estudios de MSA de los equipos RRM, ..."
17198,347042,La majorité des ménages PDI vit dans des logem...,[],fr,The majority of PDI households live in housing...,NaN,La mayoría de los hogares de PDI viven en vivi...


In [5]:
col = "gender"
for df in [df_train, df_val, df_test]:
    df[col] = df[col].apply(lambda x: list(sorted(list(set(literal_eval(x))))))

In [6]:
# df_train.loc[df_train["lang"].eq("en"),
#                 "translation_en"] = df_train.loc[
#                     df_train["lang"].eq("en"), "excerpt"]
# ##
# df_train.loc[df_train["lang"].eq("fr"),
#                 "translation_fr"] = df_train.loc[
#                     df_train["lang"].eq("fr"), "excerpt"]
# ##
# df_train.loc[df_train["lang"].eq("es"),
#                 "translation_es"] = df_train.loc[
#                     df_train["lang"].eq("es"), "excerpt"]

In [7]:
male_kw_en = [
    'man', 'men', 'boy', 'boys', 'male', 'males', "him", "his", "son", "sons",
    "father", "fathers"
]
female_kw_en = [
    'woman', 'women', 'girl', 'girls', 'female', 'females', 'mother',
    'mothers', "pregnancy", "pregnancies", "pregnant", "pregnants", "she",
    "her", "hers", "reproductive", "maternal", "daughter", "daughters",
    "childbearing", "feminicide", "feminicides", "femicide", "femicides",
    "vbg", "gbv", "sgbv", "fgm", "srh", "lactating", "menstrual", "contraceptive",
    "fetus", "foetation", "foetus",
]
##
male_kw_fr = [
    "homme", "hommes", "garçon", "garçons", "masculin", "masculins", "lui", 
]
female_kw_fr = [
    "femme", "femmes", "vbg", "maternelle", "maternelles", "vénézuélienne",
    "vénézuéliennes", "mère", "mères", "sexiste", "grossesse", "grossesses",
    "elle", "elles" "fille", "filles", "reproduction", "reproductions", 
    "gbv", "migrante", "migrantes", "survivante", "survivantes", "allaitante"
    "allaitantes", "handicapée", "handicapées", "srh", "fgm", "féminicide",
    "féminicides", "affectées", "réfugiée", "réfugiées", "sgbv", "mariée",
    "mariées", "féminine", "féminines", "violée", "violées", "sis", "menstruelle",
    "menstruelles", "péruvienne", "péruviennes", 
]
##
male_kw_es = [
    "hombre", "hombres", "viajero", "viajeros", "masculino", "masculinos",
    "venezolano", "venezolanos", "macho", "machos", "testigo", "masculina",
    "niño", "niños", 
]
female_kw_es = [
    "mujere", "mujeres", "vbg", "venezolana", "venezolanas", "materna", "maternas", 
    "embarazada", "embarazadas", "madre", "madres", "embarazo", "embarazos", "supervivencia",
    "supervivencias", "mujer", "mujers", "gbv", "fgm", "lactante", "lactantes", "lideresa",
    "lideresas", "hija", "hijas", "feminicidio", "feminicidios", "femenina", "femeninas",
    "anticonceptivo", "anticonceptivos", "srh", "refugiada", "refugiadas", "peruana", "peruanas",
    "niñas", "niña", "embarazo", "embarazos", "gestantes", "gestante", "menstrual", "menstruals",
    "sgbv", "menstruale", "menstruales", "vulnerabilidade", "vulnerabilidades", "desplazada",
    "desplazadas", "casada", "casadas", "prenatale", "prenatales", "femicidio", "femicidios",
    
]

In [8]:
# male_kw = set(male_kw_en + male_kw_fr + male_kw_es)
# female_kw = set(female_kw_en + female_kw_fr + female_kw_es)

In [9]:
def preprocess_and_tokenize(doc):
    # remove preceeding dates
    try:
        doc = re.sub("^\[.+\]", " ", doc).strip()
    except:
        print(type(doc), doc)
        raise
    doc = re.sub("^\(.+\)", " ", doc).strip()
    # spaces btw numbers and words
    doc = re.sub('(\d+(\.\d+)?)', r' \1 ', doc).strip()
    # tokenize
    words = word_tokenize(doc)
    # lower and remove non-words
    words = [word.lower() for word in words if word.isalpha()]
    return words

In [10]:
def pred_gender(row):
    words = preprocess_and_tokenize(row['excerpt'])
    if row['lang']=='en':
        female_kw = female_kw_en
        male_kw = male_kw_en
    elif row['lang']=='fr':
        female_kw = female_kw_fr
        male_kw = male_kw_fr
    elif row['lang']=='es':
        female_kw = female_kw_es
        male_kw = male_kw_es
    else:
        raise
    lbls = []
    if any(word in words for word in female_kw):
        lbls.append("Female")
    if any(word in words for word in male_kw):
        lbls.append("Male")
    return lbls

In [11]:
df_train["gender_kw_pred"] = df_train.progress_apply(pred_gender, axis=1)

  0%|          | 0/126323 [00:00<?, ?it/s]

In [12]:
df_val["gender_kw_pred"] = df_val.progress_apply(pred_gender, axis=1)

  0%|          | 0/14425 [00:00<?, ?it/s]

In [13]:
(df_train["gender_kw_pred"] == df_train["gender"]).sum(), df_train.shape

(107757, (126323, 8))

In [14]:
(df_val["gender_kw_pred"] == df_val["gender"]).sum(), df_val.shape

(12055, (14425, 8))

In [15]:
df_train.to_csv("train_0.7.1_kw_gender.csv", index=False)
df_val.to_csv("val_0.7.1_kw_gender.csv", index=False)

In [16]:
len(df_train), len(df_val)

(126323, 14425)